# Image Captioning with Transformers
Sub PJT 2에서는 Sub PJT 3에서 사용할 GPT2를 사용하기에 앞서, transformer 모델에 더 익숙해지고, 이를 활용하여 이미지 캡셔닝(image captioning) 모델을 구현합니다.
본 실습에서는 transformer decoder의 핵심 구성 요소를 구현해야 하기 때문에 Sub PJT 1과 2 명세서에 설명한 transformer에 대한 정확한 이해가 선행되어야 합니다.
작은 데이터로 모델을 학습시킬 것이 때문에, overfitting 현상이 일어나는 점 참고 바랍니다.

## 준비
먼저 필요한 모듈들을 import하고, 에러를 측정하는 함수를 정의합니다.

In [ ]:
# Setup cell.
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from lib.transformer_layers import *
from lib.captioning_solver_transformer import CaptioningSolverTransformer
from lib.classifiers.transformer import CaptioningTransformer
from lib.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from lib.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # Set default size of plots.
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


import numpy as np
import matplotlib.pyplot as plt

from lib.transformer_layers import *
from lib.captioning_solver_transformer import CaptioningSolverTransformer
from lib.classifiers.transformer import CaptioningTransformer
from lib.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from lib.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # Set default size of plots.
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

# 상대적인 에러 측정 함수
def rel_error(x, y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# COCO Dataset
본 실습에서는 COCO dataset을 사용하여 transformer를 학습시킵니다.
앞서 언급한대로, 데이터셋은 기존 COCO Dataset의 일부로, 그 크기가 매우 작습니다.

## 데이터셋 다운로드
./lib/datasets/ 디렉토리로 이동하여 get_datasets.sh 실행파일을 실행시켜 데이터를 다운로드합니다.

In [ ]:
%cd ./lib/datasets/
!bash get_datasets.sh
%cd ../../

## 데이터 형태 출력하기
아래 코드를 실행하여 다운로드한 데이터 형태를 출력해 봅니다.
본 실험에서는 이미지의 특징 벡터(feature vector)는 데이터로 이미 주어지고, 주어진 특징 벡터를 바탕으로 텍스트를 생성하는 transformer를 학습시킵니다.

In [ ]:

# COCO data를 디스크로 부터 dictionary 형태로 load
data = load_coco_data(pca_features=True)

# data dictionary의 key와 value를 출
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

# Transformer
이미지 캡셔닝 모델 구현에 앞서, transformer ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) 에 대해서 복습해 봅니다.
Transformer는 컴퓨터 비전과 자연어 처리 등의 다양한 딥러닝 모델들에 적용되고, 특히 Sub PJT3에서 사용할 언어 모델인 GPT2에도 사용됩니다.
본 실습에서는 이미지 캡셔닝 모델 관점에서 transformer 사용되지만, 실습을 통해 transformer의 가장 핵심 요소인 "attention"이라는 개념을 꼭 숙지 해야 합니다.

# Transformer: Multi-Headed Attention

### Dot-Product Attention

Attention 연산은 query $q\in\mathbb{R}^d$, value vectors $\{v_1,\dots,v_n\}, v_i\in\mathbb{R}^d$, 그리고 key vectors $\{k_1,\dots,k_n\}, k_i \in \mathbb{R}^d$ 가 주어졌을 때 아래와 같이 연산됩니다:

\begin{align}
c = \sum_{i=1}^{n} v_i \alpha_i &\alpha_i = \frac{\exp(k_i^\top q)}{\sum_{j=1}^{n} \exp(k_j^\top q)} \\
\end{align}

이때 $\alpha_i$는 "attention weights" 이라고 불리고, 이는 key와 query 값의 내적과 softmax 연산을 통해서 구할 수 있습니다.
최종 출력 $c\in\mathbb{R}^d$ 는 value vector들의 weighted average로 표현됩니다.

### Self-Attention
Transformer에서는 입력 $X \in \mathbb{R}^{\ell \times d}$ ($\ell$ 은 시퀀스 길이)로 부터 key, value, query를 얻어서 self-attention을 구현합니다. 구체적으로, 아래와 같이 입력 $X$ key, value, query로 매핑시켜주는 $V,K,Q \in \mathbb{R}^{d\times d}$ 파라미터를 학습합니다:

\begin{align}
v_i = Vx_i\ \ i \in \{1,\dots,\ell\}\\
k_i = Kx_i\ \ i \in \{1,\dots,\ell\}\\
q_i = Qx_i\ \ i \in \{1,\dots,\ell\}
\end{align}

### Multi-Headed Scaled Dot-Product Attention
Multi-headed attention의 경우, 각 head 별로 $V,K,Q \in \mathbb{R}^{d\times d}$ 파라미터를 학습합니다. 하나가 아니라 "multi" head 통해, 모델은 입력의 더욱 다양한 영역에 attend를 하여 풍부한 표현력을 가질 수 있습니다.
$h$ 를 head의 개수라고 하고, $Y_i$ 를 head $i$ attention output이라고 할 때, 각 head $i$ 별로 $Q_i$, $K_i$, $V_i$를 학습한다고 이해할 수 있습니다.
Single-headed attention을 할때와 동일한 computation complexity를 갖기 위해서 각 파라미터의 디멘젼 $Q_i \in \mathbb{R}^{d\times d/h}$, $K_i \in \mathbb{R}^{d\times d/h}$, $V_i \in \mathbb{R}^{d\times d/h}$로 설정해 줍니다. Scaling term $\frac{1}{\sqrt{d/h}}$ 를 위에서 정의한 attention에 넣어주면, 아래와 같이 수식을 작성할 수 있습니다:

\begin{equation} \label{qkv_eqn}
Y_i = \text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)(XV_i)
\end{equation}

이때 $Y_i\in\mathbb{R}^{\ell \times d/h}$ 이고, $\ell$ 은 시퀀스 길이 입니.

본 실습에서는 아래와 같이 attention map에 dropout을 적용합니다:

\begin{equation} \label{qkvdropout_eqn}
Y_i = \text{dropout}\bigg(\text{softmax}\bigg(\frac{(XQ_i)(XK_i)^\top}{\sqrt{d/h}}\bigg)\bigg)(XV_i)
\end{equation}

최종적으로 각 head의 self-attention 결과를 concatenate한 뒤 linear transformation을 거쳐서 최종 결과물로 출력됩니다:

\begin{equation}
Y = [Y_1;\dots;Y_h]A
\end{equation}

, $A \in\mathbb{R}^{d\times d}$, $[Y_1;\dots;Y_h]\in\mathbb{R}^{\ell \times d}$.

### Req 2-1: multi-head scaled dot-product attention 구현
`lib/transformer_layers.py` 에 있는 `MultiHeadAttention` 클래스 내의 multi-head scaled dot-product attention을 구현합니다.
아래는 구현 결과를 검증하기 위한 코드입니다. **본 코드에서 측정하는 상대적 에러는 `e-2`보다 작아야 합니다.**

In [ ]:
torch.manual_seed(123)

# Choose dimensions such that they are all unique for easier debugging:
# Specifically, the following values correspond to N=1, H=2, T=3, E//H=4, and E=8.
batch_size = 1
sequence_length = 3
embed_dim = 8
attn = MultiHeadAttention(embed_dim, num_heads=2)

# Self-attention.
data = torch.randn(batch_size, sequence_length, embed_dim)
self_attn_output = attn(query=data, key=data, value=data)

# Masked self-attention.
mask = torch.randn(sequence_length, sequence_length) < 0.5
masked_self_attn_output = attn(query=data, key=data, value=data, attn_mask=mask)

# Attention using two inputs.
other_data = torch.randn(batch_size, sequence_length, embed_dim)
attn_output = attn(query=data, key=other_data, value=other_data)

expected_self_attn_output = np.asarray([[[ 0.2045, -0.2835,  0.3818,  0.0162,  0.067,   0.0391,  0.2474, -0.1704],
  [ 0.3565, -0.4814,  0.1685, -0.1636, -0.1559,  0.164,   0.0828, -0.1876],
  [ 0.3478, -0.4419,  0.2334, -0.2082, -0.0563,  0.1624,  0.1341, -0.2085]]])

expected_masked_self_attn_output = np.asarray([[[ 0.6067, -1.4054, -0.3671, -0.8228, -0.2016,  0.9012,  0.1293, -0.8373],
  [ 0.3812, -0.8076,  0.1075, -0.4247, -0.0804,  0.4659,  0.1347, -0.3882],
  [ 0.3994, -0.4801,  0.1653, -0.2918, -0.1215,  0.1917,  0.0911, -0.2506]]])

expected_attn_output = np.asarray([[[-0.2223, -0.08,    0.8046,  0.5508,  0.0229, -0.0801,  0.375,  -0.0188],
  [-0.2504, -0.0431,  0.8708,  0.5467,  0.0791, -0.2084,  0.4019,  0.0505],
  [-0.0887, -0.0811,  0.7949,  0.2688,  0.0768, -0.5126,  0.3118,  0.2228]]])


print('self_attn_output error: ', rel_error(expected_self_attn_output, self_attn_output.detach().numpy()))
print('masked_self_attn_output error: ', rel_error(expected_masked_self_attn_output, masked_self_attn_output.detach().numpy()))
print('attn_output error: ', rel_error(expected_attn_output, attn_output.detach().numpy()))

# Positional Encoding
Transformer는 attention 매커니즘에 의해 입력 내에서 다양한 부분을 attend할 수 있지만, 입력 token의 순서를 알 수 있는 방법은 없습니다.
하지만 자연어 처리와 같은 태스크에서는 token의 순서가 매우 중요합니다.
이를 해결하고자 transformer의 저자들은 각 단어 token의 순서를 지정해 줄 수 있는 positional encoding을 더해줬습니다.

Positional encoding은 $P \in \mathbb{R}^{l\times d}$ matrix로 정의 하고 matrix의 값은 아래와 같이 정의됩니다.

$
P_{ij} =
\begin{cases}
\text{sin}\left(i \cdot 10000^{-\frac{j}{d}}\right) & \text{if j is even} \\
\text{cos}\left(i \cdot 10000^{-\frac{(j-1)}{d}}\right) & \text{otherwise} \\
\end{cases}
$

따라서 transformer에 입력값 $X \in \mathbb{R}^{l\times d}$ 를 바로 입력하는 대신, positional encoding이 더해진 $X + P$를 모델에 입력합니다.

### Req 2-2: PositionalEncoding 구현
`lib/transformer_layers.py` 에 있는 `PositionalEncoding` 을 구현합니다.
아래는 구현 결과를 검증하기 위한 코드입니다. **본 코드에서 측정하는 상대적 에러는 `e-3`보다 작아야 합니다.**

In [ ]:
torch.manual_seed(123)

batch_size = 1
sequence_length = 2
embed_dim = 6
data = torch.randn(batch_size, sequence_length, embed_dim)

pos_encoder = PositionalEncoding(embed_dim)
output = pos_encoder(data)

expected_pe_output = np.asarray([[[-0.1239,  1.2448, -0.4107,  0.,     -1.3299,  1.3436],
  [-0.,     -0.2386, 0.4114,  0.9893,  0.2361,  0.    ]]])

print('pe_output error: ', rel_error(expected_pe_output, output.detach().numpy()))

# Transformer for Image Captioning
앞서 구현한 것들을 모두 조합하여, 드디어 transformer 기반의 이미지 캡셔닝 모델을 구현합니다. `lib/classifiers/transformer.py`을 열고, `CaptioningTransformer` 클래스를 살펴봅니다.

### Req 2-3: `CaptioningTransformer` 클래스의 `forward` 함수 구현
함수를 구현 한 뒤 아래 코드를 실행하여 작은 test 데이터에 대해서 평가를 해봅니다.
**본 코드에서 측정하는 에러는 `e-5`보다 작아야 합니다.**

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

N, D, W = 4, 20, 30
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 3

# transformer 인스턴스 생성
transformer = CaptioningTransformer(
    word_to_idx,
    input_dim=D,
    wordvec_dim=W,
    num_heads=2,
    num_layers=2,
    max_length=30
)

# Set all model parameters to fixed values
for p in transformer.parameters():
    p.data = torch.tensor(np.linspace(-1.4, 1.3, num=p.numel()).reshape(*p.shape))

features = torch.tensor(np.linspace(-1.5, 0.3, num=(N * D)).reshape(N, D))
captions = torch.tensor((np.arange(N * T) % V).reshape(N, T))

scores = transformer(features, captions)
expected_scores = np.asarray([[[-16.9532,   4.8261,  26.6054],
         [-17.1033,   4.6906,  26.4844],
         [-15.0708,   4.1108,  23.2924]],
        [[-17.1767,   4.5897,  26.3562],
         [-15.6017,   4.8693,  25.3403],
         [-15.1028,   4.6905,  24.4839]],
        [[-17.2172,   4.7701,  26.7574],
         [-16.6755,   4.8500,  26.3754],
         [-17.2172,   4.7701,  26.7574]],
        [[-16.3669,   4.1602,  24.6872],
         [-16.7897,   4.3467,  25.4831],
         [-17.0103,   4.7775,  26.5652]]])
print('scores error: ', rel_error(expected_scores, scores.detach().numpy()))

# Overfit Transformer Captioning Model on Small Data

### Req 2-4: `CaptioningSolverTransformer` 클래스의 `transformer_temporal_softmax_loss` 함수 구현
`lib/captioning_solver_transformer.py`을 열고, `CaptioningSolverTransformer` 클래스 내에서 loss를 계산하는 코드를 구현합니다.

### Req 2-5: `CaptioningSolverTransformer` 클래스의 `_step` 함수 구현
`lib/captioning_solver_transformer.py`을 열고, `CaptioningSolverTransformer` 클래스 내에서 single gradient update를 하는 코드를 구현합니다.


구현을 완료한 뒤 아래 코드를 실행하여 작은 데이터셋에 대해서 transformer 기반의 이미지 캡셔닝 모델을 오버피팅 시켜봅니다.
학습 데이터에 대해서 loss 그래프는 감소하는 형태를 띄어야 합니다.

In [ ]:
torch.manual_seed(231)
np.random.seed(231)

data = load_coco_data(max_train=50)

transformer = CaptioningTransformer(
          word_to_idx=data['word_to_idx'],
          input_dim=data['train_features'].shape[1],
          wordvec_dim=256,
          num_heads=2,
          num_layers=2,
          max_length=30
        )


transformer_solver = CaptioningSolverTransformer(transformer, data, idx_to_word=data['idx_to_word'],
           num_epochs=100,
           batch_size=25,
           learning_rate=0.001,
           verbose=True, print_every=10,
         )

transformer_solver.train()

# Plot the training losses.
plt.plot(transformer_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

학습을 완료한 뒤 final loss 값을 프린트 합니다.
**본 코드에서 측정하는 에러는 `0.03`보다 작아야 합니다.**

In [ ]:
print('Final loss: ', transformer_solver.loss_history[-1])

# Transformer Sampling at Test Time

아래 작성된 sampling 코드를 실행하여 학습이 완료된 transformer 모델로 이미지 캡셔닝을 수행해 봅니다.
학습 데이터에 대해서는 좋은 결과를 보이지만, 작은 데이터셋에 대해서 학습이 되었기 때문에 validation 데이터에 대해서는 성능이 좋지 못함을 확인 할 수 있습니다.

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = transformer.sample(features, max_length=30)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img)
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()